In [1]:
import Levenshtein
import numpy as np
import time
import pandas as pd
from nltk.util import ngrams

In [2]:
# Record the consumed time when ran a code block.
class Timer(object):
    def __init__(self, block_name, prefix="----->"):
        self.block_name = block_name
        self.prefix = prefix

    def __enter__(self):
        print(self.prefix+"Started '"+self.block_name+"' block...")
        self.time_start = time.time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        elapsed_time = round(time.time() - self.time_start, 2)
        print(self.prefix+"Finished '"+self.block_name+"' block, time used:", str(elapsed_time)+"s.")

In [3]:
def edit_set_ratio(a, b):
    return Levenshtein.setratio(a.split(), b.split())

In [4]:
def edit_set_ratio1(a, b):
    return Levenshtein.setratio(a, b)

In [5]:
a = ["he she me"] * 5000000
b = ["she hate he"] * 5000000
# print(edit_set_ratio(a[0], b[0]))
# print(edit_set_ratio(a[1], b[1]))
with Timer("veced"):
    print(np.vectorize(edit_set_ratio)(a, b))

----->Started 'veced' block...
[0.77777778 0.77777778 0.77777778 ... 0.77777778 0.77777778 0.77777778]
----->Finished 'veced' block, time used: 14.3s.


In [6]:
t = pd.DataFrame({
    "a": [["he", "she", "me"]] * 5000000,
    "b": [["he", "she", "me"]] * 5000000
})
# with Timer("non-veced"):
#     print(t.apply(lambda x: edit_set_ratio1(x["a"], x["b"]), axis=1))

----->Started 'non-veced' block...
----->Finished 'non-veced' block, time used: 2.05s.


KeyboardInterrupt: 

In [7]:
a = [1, 2, 3]
b = [4, 5, 6]
print(list(map(max, a, b)))

[4, 5, 6]


In [28]:
def _unigrams(words):
    """
        Input: a list of words, e.g., ["I", "am", "Denny"]
        Output: a list of unigram
    """
    assert type(words) == list
    return words

# One line time: 5.04 µs ± 272 ns
def _bigrams(words, join_string='_', skip=0):
    """
      Input: a list of words, e.g., ["I", "am", "Denny"]
      Output: a list of bigram, e.g., ["I_am", "am_Denny"]
      I use _ as join_string for this example.
    """
    assert type(words) == list
    L = len(words)
    if L > 1:
        lst = []
        for i in range(L-1):
            for k in range(1,skip+2):
                if i+k < L:
                    lst.append(join_string.join([words[i], words[i+k]]) )
    else:
        # set it as unigram
        lst = _unigrams(words)
    return lst


# One line time: 5.89 µs ± 196 ns
def _trigrams(words, join_string='_', skip=0):
    """
      Input: a list of words, e.g., ["I", "am", "Denny"]
      Output: a list of trigram, e.g., ["I_am_Denny"]
      I use _ as join_string for this example.
    """
    assert type(words) == list
    L = len(words)
    if L > 2:
        lst = []
        for i in range(L-2):
            for k1 in range(1,skip+2):
                for k2 in range(1,skip+2):
                    if i+k1 < L and i+k1+k2 < L:
                        lst.append(join_string.join([words[i], words[i+k1], words[i+k1+k2]]) )
    else:
        # set it as bigram
        lst = _bigrams(words, join_string, skip)
    return lst

def unichars(text):
    return _unigrams(list(text))

def bichars(text):
    return _bigrams(list(text))

def trichars(text):
    return _trigrams(list(text))

def unigrams(text):
    return _unigrams(text.split())

def bigrams(text):
    return _bigrams(text.split())

def trigrams(text):
    return _trigrams(text.split())

In [35]:
def unigrams(text):
    return text.split()
a = ["i loke you"] * 10
m = np.vectorize(f)
print(m(a))

['i loke you' 'i loke you' 'i loke you' 'i loke you' 'i loke you'
 'i loke you' 'i loke you' 'i loke you' 'i loke you' 'i loke you']
